In [2]:
# https://www.kaggle.com/joshmcadams/oranges-vs-grapefruit/data

In [91]:
import keras
from keras.datasets import imdb
from keras import models
from keras import layers
from keras import optimizers
import numpy as np
import pandas as pd
import math
import random

In [68]:
rawCitrusData = pd.read_csv('citrus.csv', sep=',',header=1).values
np.random.shuffle(rawCitrusData)
print(rawCitrusData)

[['orange' 10.28 179.44 164 79 6]
 ['grapefruit' 13.53 229.35 145 77 14]
 ['grapefruit' 9.74 170.5 168 74 26]
 ...
 ['orange' 9.05 161.49 159 76 8]
 ['grapefruit' 11.47 197.81 165 81 12]
 ['orange' 8.7 155.79 144 90 4]]


In [69]:
# The idea will be to use binary arrays to feed the layers.
citrusDataAsNumbers = [] # Orange 0; Grapefruit 1
for item in rawCitrusData:
    type = 0
    if item[0] == 'grapefruit':
        type = 1
    newItem = np.append(np.array([type]), item[1:])
    citrusDataAsNumbers.append(newItem)
citrusDataAsNumbers = np.array(citrusDataAsNumbers)

for x in citrusDataAsNumbers:
    x[1] = math.ceil(x[1] * 100)
    x[2] = math.ceil(x[2] * 100)
    
    x[1] = x[1]//50
    x[2] = x[2]//50
    x[3] = x[3]//10
    x[4] = x[4]//10
    x[5] = x[5]//10

print(citrusDataAsNumbers)


[[0 20 358 16 7 0]
 [1 27 458 14 7 1]
 [1 19 341 16 7 2]
 ...
 [0 18 322 15 7 0]
 [1 22 395 16 8 1]
 [0 17 311 14 9 0]]


In [70]:
modelInputData = np.zeros((len(citrusDataAsNumbers), 4000))
for index, value in enumerate(citrusDataAsNumbers):
    modelInputData[index][0 + value[1]] = 1
    modelInputData[index][1000 + value[2]] = 1
    modelInputData[index][2000 + value[3]] = 1
    modelInputData[index][3000 + value[4]] = 1
    
print(modelInputData)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [71]:
all_x_data = modelInputData
all_y_data = np.zeros(len(citrusDataAsNumbers))

for index, value in enumerate(citrusDataAsNumbers):
    all_y_data[index] = citrusDataAsNumbers[index][0]

print(all_x_data)
print(all_y_data)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[0. 1. 1. ... 0. 1. 0.]


In [72]:
x_train = all_x_data[::2]
x_test = all_x_data[1::2]
print(len(all_x_data))
print(len(x_train))
print(len(x_test))
    
y_train = all_y_data[::2]
y_test = all_y_data[1::2]
print(len(all_y_data))
print(len(y_train))
print(len(y_test))
    

9999
5000
4999
9999
5000
4999


In [73]:
model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(4000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy',
                  metrics=['accuracy'])

# Train the network.
model.fit(x_train, y_train, epochs=4, batch_size=512)
results = model.evaluate(x_test, y_test)
print(results)

Epoch 1/4
5000/5000 [==============================] - 0s 58us/step - loss: 0.6765 - accuracy: 0.7142
Epoch 2/4
5000/5000 [==============================] - 0s 34us/step - loss: 0.6209 - accuracy: 0.8784
Epoch 3/4
5000/5000 [==============================] - 0s 34us/step - loss: 0.5598 - accuracy: 0.8990
Epoch 4/4
4999/4999 [==============================] - 0s 30us/step
[0.48154339426802406, 0.896379292011261]


In [119]:
customInputIndex = random.randint(0,4000)
x_custom = np.array([all_x_data[customInputIndex]])
custom_raw = rawCitrusData[customInputIndex]
print(x_custom)
print(custom_raw)
prediction = model.predict(x_custom)
print('It is an grapefruit, I am ' + str(math.ceil(prediction[0][0] * 100)) + '% sure.')

[[0. 0. 0. ... 0. 0. 0.]]
['orange' 7.76 141.46 154 74 9]
It is an grapefruit, I am 29% sure.
